# IN4080: obligatory assignment 1 (Autumn 2024)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Sunday, September 15 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory\_1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

## Part 1 : Tokenisation

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [1]:
from typing import List
import re

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""

    # \s+    : Matches any whitespace sequence (spaces, tabs, newlines).
    # |      : OR operator to match either whitespace or the following group.
    # ([\.,:;!\?\-\(\)\""]) : A capturing group for punctuation marks. 
    # This ensures punctuation is captured and included in the split result.
    # \      : using it before special characters allows us to treat them literally.
    tokens = re.split(r'\s+|([\.,:;!\?\-\(\)\""])', text)

    # Filtering out any empty strings resulting from consecutive matches or splitting at the start/end
    tokens = [token for token in tokens if token]

    return tokens

basic_tokenize('Pierre, who works at NR, also teaches at UiO.')

['Pierre',
 ',',
 'who',
 'works',
 'at',
 'NR',
 ',',
 'also',
 'teaches',
 'at',
 'UiO',
 '.']

To develop the above code, I consulted the documentation for the Regex library, specifically following the instructions on the following page regarding the use of the split method: https://docs.python.org/3/library/re.html#making-a-phonebook.

We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 

In [2]:
file_path = 'ndt_test_lm.txt'

with open(file_path, mode='r', encoding='utf-8') as file:
    text = file.read()

tokens = basic_tokenize(text)

# Proof that the algorithm actually works by printing the first 30 elements of the list
print(f'First thirty tokens: {tokens[:30]}')

# Number of tokens extracted
print(f'Number of tokens extracted: {len(tokens)}')

# Number of unique words (types), not considering punctuation
def count_types(a_list):
    lower_case_list = [token.lower() for token in a_list] # We turn every word into lowercase 
                                                          # to avoid any possible repetition in the types.
    punctuation_string = '.,:;!?-()"'
    punctuation_set = set(punctuation_string) # Creating a set with all the separators
    types = set()
    for element in lower_case_list:
        if element not in punctuation_set:
            types.add(element)  # Adding only unique non-punctuation elements, given that types is a set
    return len(types)
print(f'Number of types: {count_types(tokens)}')

First thirty tokens: ['Lam', 'og', 'piggvar', 'på', 'bryllupsmenyen', '|', 'Kamskjell', ',', 'piggvar', 'og', 'lammefilet', 'sto', 'på', 'menyen', 'under', 'den', 'kongelige', 'gallamiddagen', '.', 'Og', 'til', 'dessert', ':', 'Parfait', 'à', 'la', 'Mette', '-', 'Marit', '.']
Number of tokens extracted: 259220
Number of types: 27586


We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [3]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """
    Tokenizer based on the Byte-Pair Encoding algorithm.
    Note: the current implementation is limited to Latin characters (ISO-8859-1)
    """

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """
        Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached.
        """

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        # We specified which encoding to use when reading the file
        with open(train_corpus_file, encoding='iso-8859-1') as fd: 
            # We first read the corpus, split on white space, 
            # and count the occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                # get() method parameters:
                # keyname (required): The keyname of the item we want to return the value from.
                # value (pptional): A value to return if the specified key does not exist.
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1 # Dictionary {'token':count}
            print("Done")

            # We then iteratively extend the list of merge pairs 
            # until we reach the desired size. 
            # Note: to speed up the algorithm, we extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
                # print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], n:int=200) -> List[Tuple[str,str]]:
        """
        Given a set of distinct words along with their corresponding number of occurrences in the corpus, 
        returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        for word in tqdm(words) if show_progress_bar else words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                
    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge
            # that is, the one with the lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ? 

In [4]:
# File paths for training and testing corpora
train_file_path = 'ndt_train_lm.txt'
test_file_path = 'ndt_test_lm.txt'

In [5]:
# We train the tokenizer using the `ndt_train_lm.txt` corpus 
bpe_tokenizer = BPETokenizer(train_corpus_file=train_file_path, vocab_size=5000)

# We apply the BPE Tokenizer to the `ndt_test_lm.txt` corpus
with open(test_file_path, mode='r', encoding='iso-8859-1') as test_file:
    test_text = test_file.read()

tokens = list(bpe_tokenizer(test_text))  # Tokenizing the test corpus thanks to the __call__ method

# We count the number of tokens and types
num_tokens = len(tokens)             # Total number of tokens
num_types = len(set(tokens))         # Number of unique tokens (types)

print(f"Number of tokens: {num_tokens}")
print(f"Number of types: {num_types}")

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/226175 [00:00<?, ?it/s]

Number of tokens: 386753
Number of types: 4399


- **Basic Tokenizer**

Number of tokens: 259 220

Number of types: 27 586

- **BPE Tokenizer**

Number of tokens: 386 753

Number of types: 4 399

**Final considerations:**

The basic tokenizer does not split the single words, while the BPE tokenizer does, thus the highest number of tokens.

__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

**Modified BPE Tokenizer details**: I reported the above implemented 'basic_tokenize' function inside the class, and I modified the initial __init__ function so that when creating the dictionary 'vocabulary_counts', it no longer uses the split method to find the different words (tokens) from the text, but instead uses the 'basic_tokenize' function. Finally, I modified the __call__ function in the same way.

In [6]:
# -------------------------- MODIFIED BPE TOKENIZER --------------------------
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer_modified:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""

    @staticmethod                                                  # We indicate that the method is independent of any instance or class attributes.
    def basic_tokenize(text: str) -> List[str]:
        tokens = re.split(r'\s+|([\.,:;!\?\-\(\)\""])', text)
        tokens = [token for token in tokens if token]
        return tokens

    def __init__(self, train_corpus_file: str, vocab_size = 5000):
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        with open(train_corpus_file, encoding='iso-8859-1') as fd: # I specified which encoding to use when reading the file

            # We first read the corpus, split on white space, and count the
            # occurrences of each distinct word
            print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            tokens = self.basic_tokenize(text)                     # Instead of using the split method, I applied my own function
            for token in tokens:                     
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                progress_bar.update(len(most_common_pairs))
         #       print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10])
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        tokens = self.basic_tokenize(input)                              # Instead of using the split method, I applied my own function
        for token in tqdm(tokens) if show_progress_bar else tokens:
            subwords = self.tokenize_word(token)
            for subword in subwords:
                yield subword
                
    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

In [7]:
# Testing the modified BPE Tokenizer

# File paths for training and testing corpora
train_file_path = 'ndt_train_lm.txt'
test_file_path = 'ndt_test_lm.txt'

# We train the tokenizer using the `ndt_train_lm.txt` corpus 
bpe_tokenizer = BPETokenizer_modified(train_corpus_file=train_file_path, vocab_size=5000)

# We apply the BPE Tokenizer to the `ndt_test_lm.txt` corpus
with open(test_file_path, mode='r', encoding='iso-8859-1') as test_file:
    test_text = test_file.read()

tokens = list(bpe_tokenizer(test_text))  # Tokenizing the test corpus using the __call__ method

# We count the number of tokens and types
num_tokens = len(tokens)             # Total number of tokens
num_types = len(set(tokens))         # Number of unique tokens (types)

print(f"Number of tokens: {num_tokens}")
print(f"Number of types: {num_types}")

Counting word occurrences in corpus ndt_train_lm.txt...Done


  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/259294 [00:00<?, ?it/s]

Number of tokens: 392625
Number of types: 4316


- **BPE Tokenizer**

Number of tokens: 386 753

Number of types: 4 399

- **Modified BPE Tokenizer**

Number of tokens: 392 625

Number of types: 4 316

**Final considerations:**

In the original tokenizer, we observe fewer tokens but more types. This could be because the modified BPE tokenizer splits punctuation signs, which increases the total number of tokens while simultaneously reducing the number of unique ones.

__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published earlier this year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

After reading Andrej Karpathy's tweet, I decided to focus on the first four issues LLM face as a result of tokenization. These particular issues are:
- Why can't LLM spell words? **Tokenization.**
- Why can't LLM do super simple string processing tasks like reversing a string? **Tokenization.**
- Why is LLM worse at non-English languages (e.g. Japanese)? **Tokenization.**
- Why is LLM bad at simple arithmetic? **Tokenization.**

### Why can't LLM spell words?

There are many examples on the Internet where users show how LLMs are unable to spell simple words or count the number of a particular letter in a particular word. The reason for this is that LLMs don't analyze words given as prompts letter by letter, but break them up into subwords (tokens) that may consist of the entire word or parts of the word itself. This shows that LLMs don't inherently understand how to manipulate individual letters the way we do. An example would be the handling of the word *'hello*, which could be split into *'hel' + 'lo*, making spelling corrections more difficult because the LLM is working at a higher level of abstraction (tokens), not directly at the character level.

### Why can't LLM do super simple string processing tasks like reversing a string?

Again, since LLMs operate at a higher level of abstraction represented by tokens, and not at the character level, if we give the LLM a particular word as a prompt and ask the model to invert it, the LLM might split the word into different tokens and then work on inverting each individual token, producing a final result that does not match the desired output. Using again the example of the word *'hello'*, split into the tokens *'hel' + 'lo'*, the LLM might first split the first token (*'hel* --> *'leh*), then the second one (*lo* --> *ol*), and in the worst case put them together to get *'lehol'*.

### Why is LLM worse at non-English languages (e.g. Japanese)?

The vast majority of tokenization algorithms were developed with English in mind and may not work well for all languages that have a different syntactic structure. For example, languages such as Japanese and Chinese don't have spaces between words, making the tokenization task more difficult due to the additional ambiguity that requires more context to understand the role each word plays. In addition, Japanese is characterized by the use of three different vocabularies, kanji, hiragana, and katakana, all of which occur simultaneously in different sentences, creating an even more complicated field for LLM to work in, given that it shouldn't mix the different vocabularies during the tokenization process.

### Why is LLM bad at simple arithmetic?

Given that LLM are trained to predict the next token in a sequence based on probabilities derived from massive amounts of text data, they don't actually associate the token represented by *'123'* with the actual value of 123, so when we ask the LLM to add another number (like 256) to that number, the model will base its answer solely on probabilistic models, without following the actual arithmetic rules behind addition. LLM do not treat numbers as actual numbers, they treat them as any other token.

To avoid most of the above problems, it is possible to ask the LLM to solve the different requests using, for example, a Python script. The LLM is able to generate the correct syntax and structure for Python, based on patterns it has learned from large amounts of code data, allowing it to correctly handle operations such as string manipulation and different types of calculations, like addition.

## Part 2: N-gram language models

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [14]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer
 
    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
          mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        for i in tqdm(range(len(tokens))):
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                  alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()
 

    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [15]:
def __init__(self, training_corpus_file: str, tokenizer: BPETokenizer, ngram_size: int = 2, alpha_smoothing: float = 1.0):
    """Initialize the N-gram model with:
    - a file path to a training corpus to estimate the N-gram probabilities
    - an already learned BPE tokenizer
    - an N-gram size
    - a smoothing parameter (Laplace smoothing)"""
    # We initialize the parent class
    LanguageModel.__init__(self, tokenizer)
    self.ngram_size = ngram_size
    self.alpha_smoothing = alpha_smoothing  # Store the smoothing parameter

    # We build a new vocabulary including the special tokens to highlight the beginning and the end of a sentence
    self.vocab = self.tokenizer.vocab.copy()    # We make a copy of the vocabulary 
                                                # from the BPE Tokenizer implemented before
    self.vocab.append("<s>")   # Start of sentence token
    self.vocab.append("</s>")  # End of sentence token
    V = len(self.vocab)        # Vocabulary size

    # Initialize dictionaries to hold counts
    ngram_counts = {}     # Counts of n-grams
    context_counts = {}   # Counts of contexts (n-1)-grams

    with open(training_corpus_file, 'r', encoding='utf-8') as fd:

        # Iterate over each line in the corpus
        for line in fd:
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            # Tokenize the line using the BPE tokenizer
            tokens = list(self.tokenizer(line))

            # Add start and end tokens to the list of tokens
            tokens = ["<s>"] * (self.ngram_size - 1) + tokens + ["</s>"]

__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [10]:
# Libraries
import numpy as np

# Reading the 'ndt_train_class.txt' file
ndt_train_class_path = 'ndt_train_class.txt'

with open(ndt_train_class_path, mode='r', encoding='utf-8') as file:
    sentences_list = file.readlines()       # We use readlines() instead of the classic read()
                                            # in order to obtain a list where each element is a line from the file

sentences_list_without_labels = []          # A list containing sentences without the additional 'nno' and 'nob' labels
labels = []                                 # 0 = Bokmål; 1 = Nynorsk

for line in sentences_list:
    line = line.strip()
    if not line:
        continue                            # We skip empty lines
    if '\t' in line:
        sentence, label = line.split('\t')  # The label is separated by the actual sentence by \t
                                            # so we are dividing the sentence from the label
        if label == 'nno':
            labels.append(1)                # Nynorsk
        elif label == 'nob':
            labels.append(0)                # Bokmål
        else:
            print(f"Unknown label '{label}' found.")
            continue                        # Skip lines with unknown labels
        sentences_list_without_labels.append(sentence)
    else:
        print(f"Line format incorrect: '{line}'")
        continue                            # Skip lines without a tab character

# ------------------- Finding the N value -------------------
# Counting the number of different sentences in the file (N)
N = len(sentences_list_without_labels)
print(f'Number of sentences: {N}')

# ------------------- Creating the target vector -------------------
target_vector = np.array(labels)            # Vector of length N with a value of 1 if the sentence was marked as Nynorsk, 
                                            # and 0 if is was marked as Bokmål. 
print(f'Target vector shape: {target_vector.shape}')

# ------------------- Finding the V value ------------------- 
vocabulary = bpe_tokenizer.vocab
V = len(vocabulary)

print(f'Number of features: {V}')

# ------------------- Tokenizing each sentence ------------------- 
tokenized_sentences = []

for sentence in sentences_list_without_labels:
    tokens = list(bpe_tokenizer(sentence, show_progress_bar=False))
    tokenized_sentences.append(tokens)

# We nitialize an empty dictionary to keep track of the different subwords' indices
subword_to_index = {}  

# We loop through the vocabulary with both the index and the subword
for idx, subword in enumerate(vocabulary):
    subword_to_index[subword] = idx         # We assign the index to the subword

# ------------------- Creating the feature matrix ------------------- 
X = np.zeros((N, V))

for i, tokens in enumerate(tokenized_sentences):
    for token in set(tokens):               # We use set to avoid duplicate subwords in a sentence
        idx = subword_to_index.get(token)
        if idx is not None:
            X[i, idx] = 1                   # We mark the presence of the subword in the sentence

Number of sentences: 29905
Target vector shape: (29905,)
Number of features: 5056


__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [11]:
# Libraries
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear')

# Fitting the model to our data (train data)
model.fit(X, target_vector)

LogisticRegression(solver='liblinear')

__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [12]:
# Libraries
from sklearn.metrics import accuracy_score, recall_score, precision_score

# We read the 'ndt_test_class.txt' file
ndt_test_class_path = 'ndt_test_class.txt'

with open(ndt_test_class_path, mode='r', encoding='utf-8') as file:
    test_sentences_list = file.readlines()

test_sentences_list_without_labels = []
test_labels = []

for line in test_sentences_list:
    line = line.strip()
    if not line:
        continue
    if '\t' in line:
        sentence, label = line.split('\t')
        if label == 'nno':
            test_labels.append(1)  # Nynorsk
        elif label == 'nob':
            test_labels.append(0)  # Bokmål
        else:
            print(f"Unknown label '{label}' found.")
            continue
        test_sentences_list_without_labels.append(sentence)
    else:
        print(f"Line format incorrect: '{line}'")
        continue

# We convert the test labels to a numpy array
test_target_vector = np.array(test_labels)

# We tokenize the test sentences
test_tokenized_sentences = []

for sentence in test_sentences_list_without_labels:
    tokens = list(bpe_tokenizer(sentence, show_progress_bar=False))
    test_tokenized_sentences.append(tokens)

# We create the feature matrix specifically for the test set
X_test = np.zeros((len(test_sentences_list_without_labels), V))

for i, tokens in enumerate(test_tokenized_sentences):
    for token in set(tokens):
        idx = subword_to_index.get(token)
        if idx is not None:
            X_test[i, idx] = 1

# We create a variable containing the labels predictions for the test set
predictions = model.predict(X_test)

# Finally, we evaluate the model's performance
accuracy = accuracy_score(test_target_vector, predictions)
recall = recall_score(test_target_vector, predictions)
precision = precision_score(test_target_vector, predictions)

print(f'Accuracy: {accuracy:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

Accuracy: 0.9458
Recall: 0.9278
Precision: 0.9557


__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 

In [13]:
# ------------------- Accessing the coefficients -------------------
# model.coef_ is a 2D array of shape (1, n_features),
# we extract the coefficients as a 1D array
coefficients = model.coef_[0]

# ------------------- Creating the index_to_subword dictionary -------------------
# We reverse the subword_to_index dictionary previously created 
# to obtain an index_to_subword dictionary
index_to_subword = {}
for subword, idx in subword_to_index.items():
    index_to_subword[idx] = subword

# ------------------- Associating coefficients with subwords -------------------
# We create a list of (subword, coefficient) tuples
subword_coefficients = []
for idx, coef in enumerate(coefficients):
    subword = index_to_subword[idx]  # Get the subword corresponding to the index
    subword_coefficients.append((subword, coef))

# ------------------- Sorting the subwords based on coefficients -------------------
# Sort subwords by coefficient values for positive contributions (descending)
sorted_subwords_positive = sorted(subword_coefficients, key=lambda x: x[1], reverse=True)

# Sort subwords by coefficient values for negative contributions (ascending)
sorted_subwords_negative = sorted(subword_coefficients, key=lambda x: x[1])

# ------------------- Extracting and displaying the top 5 subwords -------------------
# Top 5 subwords contributing to Nynorsk classification
top_5_nynorsk = sorted_subwords_positive[:5]
print("Top 5 subwords contributing to Nynorsk classification:")
for subword, coef in top_5_nynorsk:
    print(f"Subword: '{subword}', Coefficient: {coef:.4f}")

# Top 5 subwords contributing to Bokmål classification
top_5_bokmal = sorted_subwords_negative[:5]
print("\nTop 5 subwords contributing to Bokmål classification:")
for subword, coef in top_5_bokmal:
    print(f"Subword: '{subword}', Coefficient: {coef:.4f}")

Top 5 subwords contributing to Nynorsk classification:
Subword: 'ikkje ', Coefficient: 4.3448
Subword: 'eit ', Coefficient: 3.9601
Subword: 'dei ', Coefficient: 3.6074
Subword: 'ein ', Coefficient: 3.3617
Subword: 'ane ', Coefficient: 3.0465

Top 5 subwords contributing to Bokmål classification:
Subword: 'ikke ', Coefficient: -3.2864
Subword: 'sier ', Coefficient: -3.2536
Subword: 'jeg ', Coefficient: -3.0967
Subword: 'fra ', Coefficient: -2.8605
Subword: 'Jeg ', Coefficient: -2.6171


**Final considerations:**

Not being a native Norwegian speaker, it is a challenge for me to give a proper evaluation of the different subwords that contributed most to the classification model.

However, it is immediately noticeable that the subword *'jeg'* (*'I'* in English) appears twice in the top 5 contributors to the Bokmal classification, and while this makes sense given that personal pronouns differ between the two standards, an explicit analysis of lower case only would have helped to avoid such repetition.

In general, all of the subwords that most influenced the classification model appear to be quite characteristic of the standard to which they belong, which explains why they were so influential in the actual classification process.